# DESCUENTOS PERMANENTES

In [10]:
#librerias
import pandas as pd 
import re
from def_correo_python import limpiar_correos

**Declaración de función**

In [11]:
#declaración de función
def descuentos_permanentes (input_csv,df_izzi_sms_output,df_wizz_sms_output,df_izzi_email_output,df_wizz_email_output):
    df = pd.read_csv(input_csv)
    df_izzi = df.query("TIPO_CLIENTE == 'IZZI'")
    df_wizz = df.query("TIPO_CLIENTE == 'WIZZ'")

    #conteo de REPEP para IZZI
    print("SECCIÓN DE REPEP")
    izzi_repep = df_izzi.query("FLAG_REPEP == 'Y'")
    print(f'El número de registros en REPEP en IZZI es de {len(izzi_repep)}')
    print()
    #conteo de REPEP para WIZZ
    wizz_repep = df_wizz.query("FLAG_REPEP == 'Y'")
    print(f'El número de registros en REPEP en WIZZ es de {len(wizz_repep)}')
    print()

    #eliminación de repep
    df_izzi = df_izzi[df_izzi['TIPO_CLIENTE'] !='Y']
    df_wizz = df_wizz[df_wizz['TIPO_CLIENTE'] !='Y']

    #eliminación de campos IZZI
    df_izzi_mail = df_izzi.drop(columns='TEL_CELULAR')
    df_izzi_sms = df_izzi.drop(columns=['NOMBRES','EMAIL'])
    
    #eliminación de campos WIZZ
    df_wizz_mail = df_wizz.drop(columns='TEL_CELULAR')
    df_wizz_sms = df_wizz.drop(columns=['NOMBRES','EMAIL'])

    #limpieza de bases sms IZZI
    
    #dividir la columna 'TEL_CELULAR' por espacio y conservar solo el primer número
    df_izzi_sms['TEL_CELULAR'] = df_izzi_sms['TEL_CELULAR'].str.split().str[0]
    #filtrar aquellos celulares diferentes a 10
    df_izzi_sms = df_izzi_sms[df_izzi_sms['TEL_CELULAR'].str.len() == 10]
    # Filtros de números repetidos y seriados en SMS IZZI
    print("SECCIÓN DE números válidos")
    print(f"Números válidos en SMS IZZI antes de limpieza: {len(df_izzi_sms)}")
    patron_repetidos = r'^(.)\1{9}$'  # Detecta números repetidos como 1111111111
    patron_seriados = r'^(1234567890|9876543210)$'  # Detecta números seriados comunes
    df_izzi_sms = df_izzi_sms[~df_izzi_sms['TEL_CELULAR'].str.match(f"{patron_repetidos}|{patron_seriados}", na=False)]
    print(f"Números válidos en SMS IZZI después de limpieza: {len(df_izzi_sms)}")

    #limpieza de bases SMS WIZZ

    #dividir la columna 'TEL_CELULAR' por espacio y conservar solo el primer número
    df_wizz_sms['TEL_CELULAR'] = df_wizz_sms['TEL_CELULAR'].str.split().str[0]
    #filtrar aquellos celulares diferentes a 10
    df_wizz_sms = df_izzi_sms[df_izzi_sms['TEL_CELULAR'].str.len()==10]
    print(f"Números válidos en SMS WIZZ antes de limpieza: {len(df_wizz_sms)}")
    # Filtros de números repetidos y seriados en SMS WIZZ
    df_wizz_sms = df_wizz_sms[~df_wizz_sms['TEL_CELULAR'].str.match(f"{patron_repetidos}|{patron_seriados}", na=False)]
    print(f"Números válidos en SMS WIZZ después de limpieza: {len(df_wizz_sms)}")

    #exportaciones sms
    df_izzi_sms.to_csv(df_izzi_sms_output,index= False)
    df_wizz_sms.to_csv(df_wizz_sms_output,index= False)
    #exportaciones mail
    df_izzi_mail.to_csv(df_izzi_email_output,index= False)
    df_wizz_mail.to_csv(df_wizz_email_output, index = False)



Aplicación de función

In [12]:
descuentos_permanentes(r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\Base general 29 nov.csv',
                       r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_izzi_sms.csv',
                       r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_wizz_sms.csv',
                       r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_izzi_email.csv',
                       r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_wizz_email.csv')

SECCIÓN DE REPEP
El número de registros en REPEP en IZZI es de 2295

El número de registros en REPEP en WIZZ es de 34

SECCIÓN DE números válidos
Números válidos en SMS IZZI antes de limpieza: 96581
Números válidos en SMS IZZI después de limpieza: 96499
Números válidos en SMS WIZZ antes de limpieza: 96499
Números válidos en SMS WIZZ después de limpieza: 96499


**Lista Regex**

In [13]:
wrong_words_regex = [
    r'@@',                               # Doble arroba
    r'\.\.',                             # Doble punto
    r'\.com@',                           # ".com" seguido de arroba
    r'gm?ial|gnial|dumy|notene',         # Errores tipográficos comunes,
    r'@(yopmail)',                       # Excluir solo el dominio "yopmail"
    r'(dummy|prueba|ejemplo|temporal)',  # Palabras genéricas o placeholders
    r'[\+\&:,"\' ]',                     # Caracteres especiales o espacios
    r'\bno_tiene\b|\bNO TIENE\b',        # Frases exactas (en mayúsculas y minúsculas)
    r'\b(no|sin)_correo\b',              # "no correo" o "sin correo"
    r'\b(?:notengo|noexiste|nocuentaconcorreo)\b', # Frases comunes que indican falta de correo
    r'\bcorreoprueba\b',                 # Placeholder "correoprueba"
    r'ñ',                                # Caracter "ñ"
    r'^.*clienteizzi.*$',                # Correos que inicien con cliente izzi
    r'^(.*izzi.*)@.*$',
    r'^(1\.21|1212|1122|121|12.1|1\.2)', # Correos que jueguen con la secuencia de números 1 y 2
    r'^(618)',                           # Correos que jueguen con la secuencia de números 6
    r'^(sinusuario|demientras|notengo)', # Frases comunes que indican falta de correo
    r'^[._]',                            # Correos que comienzan con punto o guion bajo
    r'^[^a-zA-Z0-9]',                    # Correos que comienzan con caracteres no alfanuméricos
    r'^0+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',# Nuevas condiciones: solo ceros antes del dominio
    r'^1@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', # Nuevas condiciones: solo "1" antes del arroba
    r'^virtual@.*',                      # Solo elimina correos que comienzan con "virtual"
    r'^(notiene|notienecorreo|confirmaciones|provisional)@.*$', #Placeholders
    r'^(sincorreo|sinnombre|na|izzi.generico)@.*$', #Placeholders
    r'^facturas[-_]?izzi.*$',                       #facturas_izzi o facturasizzi
    r'^(clientenuevo|clienteizzi|izzi|aaa|om)@.*$', #Placeholders
    r'^(lacasadelbrujo|nohaycorreo|ninguno)@.*$',   #Placeholders
    r'^(manololo|sinmail|n|ngonzalez|generico)@.*$',#Placeholders
    r'^(atencionalcliente|izzigenerico|p-ngonzalezg|a.empresarial)@.*$',#Placeholders
    r'^(ventas\d+clientes|xxx|generico\d+)@.*$',    #Placeholders
    r'^(jesus|jose)@.*$|^sincorreo\d+@.*$',         #Nombres de tecnicos
    r'^(correo|123456|sin.correo|no.se.lo.sabe|wizzgenerico)@.*$',#Placeholders
    r'^(1234123|1.g|001|no.tengo|no.tiene|no|)@.*$',#Placeholders
    r'^(abc|abc123|atc|sincontacto|iz.z.ot.el.e)@.*$', #Placeholders
    r'^(no_tienecorreo|no_se_me_ocurrio_otro|no.tiene_correo)@.*$', #Placeholders
    r'^(no_tiene_correo|no.aplica|no.maneja.correo)@.*$', #Placeholders
    r'^(noasignacorreo|nobrinda|nocorreo|nocorreoelectronico)@.*$', #Placeholders
    r'^(nocuenta[\w-]+|nodejo[\w-]+|noda[\w-]+|nodesea[\w-]+)@', #No cuenta
    r'([\w.-]*notiene[\w.-]*@|[\w.-]+notiene[\w.-]*)', #"nocuenta" al inicio del @ o después 
    r'([\w.-]*@notiene[\w.-]*)',                    #"nocuenta" después del @
    r'^123@',                                       #Empieza con 123@ 
    r'^[^@]*$',                                     #No contiene @
    r'\.$',                                         #Termina con un punto final
    r'@.*@',                                        #Múltiples @ no consecutivos
    r"^[a-zA-Z0-9]{1,2}@[\w.-]+\.[a-zA-Z]{2,}$",    #1 o 2 dígitos antes del @
    r'^z+@',                                        #correos que solo contengan z antes del @
                                    

]

In [14]:
limpiar_correos(r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_izzi_email.csv',
                r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_izzi_email_limpia.csv',
                r'C:\Documentos\Trabajo\DB\DESCUENTOS PERMANENTES\DB BI\Noviembre\2base\bd2_izzi_email_correos_eliminados.csv',
                wrong_words_regex)

NameError: name 'pd' is not defined